<a href="https://colab.research.google.com/github/tomonari-masada/course2022-sml/blob/main/11_clustering_(example).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語の単語ベクトルをクラスタリング

* 使用するデータは下記の場所にあるもの

 * https://github.com/shiroyagicorp/japanese-word2vec-model-builder

* word2vecの技術を使って、日本語の単語が50次元のベクトルで表現されている。
 * word2vecそのものについては、ここでは説明しません。
* 今回はこの単語ベクトルをクラスタリングして、意味の近い単語が同じクラスタに属しているかをチェックする。

## 1) wgetコマンドでダウンロードし、unzipで解凍する

In [1]:
!if [ ! -e latest-ja-word2vec-gensim-model.zip ]; then wget http://public.shiroyagi.s3.amazonaws.com/latest-ja-word2vec-gensim-model.zip ; fi

--2022-07-15 03:27:05--  http://public.shiroyagi.s3.amazonaws.com/latest-ja-word2vec-gensim-model.zip
Resolving public.shiroyagi.s3.amazonaws.com (public.shiroyagi.s3.amazonaws.com)... 52.219.0.173
Connecting to public.shiroyagi.s3.amazonaws.com (public.shiroyagi.s3.amazonaws.com)|52.219.0.173|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 132936751 (127M) [application/zip]
Saving to: ‘latest-ja-word2vec-gensim-model.zip’

latest-ja-word2vec- 100%[===================>] 126.78M  60.2MB/s    in 2.1s    

2022-07-15 03:27:07 (60.2 MB/s) - ‘latest-ja-word2vec-gensim-model.zip’ saved [132936751/132936751]



In [2]:
!unzip -u latest-ja-word2vec-gensim-model.zip
#!unzip -u /content/drive/MyDrive/data/latest-ja-word2vec-gensim-model.zip

Archive:  latest-ja-word2vec-gensim-model.zip
  inflating: word2vec.gensim.model   
  inflating: word2vec.gensim.model.syn1neg.npy  
  inflating: word2vec.gensim.model.wv.syn0.npy  


## 2) ライブラリgensimを使ってデータを読み込む

In [3]:
from gensim.models.word2vec import Word2Vec

model_path = 'word2vec.gensim.model'
model = Word2Vec.load(model_path)

* 単語ベクトルの次元を確認する



In [4]:
print(model.vector_size)

50


* 単語ベクトルデータを変数名wvで参照することにする


In [5]:
wv = model.wv

* 単語リストと、対応する単語ベクトルのリストを、作成する

In [6]:
words = list()
vectors = list()
for word in wv.vocab:
  words.append(word)
  vectors.append(wv.word_vec(word))

* 単語数を確認する



In [7]:
print(len(words))

335476


* 最初の単語とその単語ベクトルを確認する



In [8]:
print(words[0])
print(vectors[0])

『
[-0.16400108 -0.17862815  0.0580783  -0.24007608  0.17230852 -0.07751795
 -0.11018773 -0.08525417 -0.01558867 -0.14766072 -0.00633486 -0.01630188
  0.00647159 -0.17860252 -0.01816674 -0.05038163  0.08417128  0.04350499
 -0.19731551 -0.09599385  0.2745512  -0.22891845  0.142788   -0.08023094
 -0.23000377  0.00842183  0.15071994  0.25936294  0.10318379  0.17875645
 -0.31201744 -0.03366571 -0.08143289  0.02155716 -0.02921231  0.0362477
 -0.18767731 -0.10514667 -0.012625   -0.03388035  0.10886643 -0.1074557
 -0.03668041 -0.03183161 -0.14188181 -0.27481845 -0.12782158 -0.02111848
 -0.14041357  0.23952545]


## 3) NumPyの配列に変換する

In [9]:
import numpy as np

words = np.array(words)
vectors = np.array(vectors)

In [10]:
print(words)

['『' 'GO' '!' ... 'スケクシス' 'ｼﾞﾑ' 'パヴリチェンコ']


In [11]:
print(vectors)

[[-0.16400108 -0.17862815  0.0580783  ... -0.02111848 -0.14041357
   0.23952545]
 [ 0.01737987 -0.32058007 -0.23378702 ...  0.13550399  0.0736375
  -0.14032933]
 [ 0.08531253 -0.31538385 -0.09230007 ...  0.17875628  0.11313058
  -0.24140361]
 ...
 [ 0.00092179 -0.07699777  0.14290373 ...  0.09500334 -0.06761228
   0.24790283]
 [ 0.04340099 -0.0260367   0.09188872 ...  0.16940522  0.00322533
  -0.20196445]
 [ 0.07351335  0.15455413  0.08450193 ... -0.26947773  0.04289696
  -0.09446784]]


* 「日本」という単語に最も近い10個の単語を表示させてみる。


In [12]:
vec_jpn = np.array(wv.word_vec('日本'))
print(vec_jpn)
indices = np.argsort(np.linalg.norm(vectors - vec_jpn, axis=1))
print(words[indices[1:11]])

[-0.08019581 -0.10632093  0.14115061  0.08310905 -0.00913872  0.04414612
 -0.11600631 -0.35167393  0.04924094  0.07659911  0.07906264  0.02693196
  0.03977293  0.19168071 -0.16376053 -0.01040754 -0.13442372  0.20567688
 -0.04850557 -0.22208223  0.03354846 -0.18884456 -0.00623878 -0.12064678
 -0.01325125  0.02779626  0.02936541  0.06843039  0.11489733  0.07094222
  0.14459857 -0.12487161  0.03558656  0.41905314 -0.15021588 -0.12890314
  0.04594489 -0.28921965  0.15588439 -0.01789199 -0.2866291  -0.1415011
  0.057166   -0.19496612  0.09121747  0.01317389 -0.0359673   0.16124834
 -0.03846167 -0.06273678]
['韓国' '台湾' '日本国内' '欧米' '日本国外' '台湾出身' '中華圏' '日本の経済' '日本の歴史' '韓国国内']


## 4) k-meansで単語ベクトルをクラスタリングする

* かなり時間がかかるので、待つ。
* 得られたクラスタの重心はCSVファイルとして保存しておく。
* クラスタ数は変更してよい。

* クラスタ数の設定



In [13]:
n_clusters = 100

* k-平均法によるクラスタリングを実行。
 * かなり時間がかかるので、待つ。

In [ ]:
from sklearn.cluster import KMeans

#kmeans = KMeans(n_clusters=n_clusters, random_state=123)
#kmeans.fit(vectors)

* クラスタリングの結果をcsvファイルとして保存。

In [ ]:
#np.savetxt('cluster_centers_{:d}.csv'.format(n_clusters), kmeans.cluster_centers_, delimiter=',')
#centers = kmeans.cluster_centers_

* k-平均法を実行するのではなく、クラスタの重心をファイルから読み込むときは、下のセルを実行。
 * パスは適当に書き換える。


In [ ]:
centers = np.loadtxt('/content/drive/MyDrive/2022Courses/sml/cluster_centers_100.csv', delimiter=',')

In [ ]:
center = centers[10]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
print(words[indices[:20]])

# 課題11

* k-平均法で単語ベクトルをクラスタリングする。
* いくつかのクラスタについて、クラスタの重心に近い単語どうしの意味が似ているかどうか、チェックする。
* いくつかのクラスタについて、クラスタの重心から遠い単語がどのような単語になっているかをチェックする。
* 極端にサイズの大きなクラスタや、逆に、極端にサイズの小さなクラスタができていないか、チェックする。
 * そして、そういった極端なサイズのクラスタに属する単語がどのようになっているかを調べる。
* k-平均法以外のクラスタリング手法でも、同じような調査をおこなってみる。




---



## A) 階層的クラスタリングの計算量
* Wikipediaのエントリにあるとおり、階層的クラスタリングは計算量が大きく、規模の大きなデータには向かない。
 * https://en.wikipedia.org/wiki/Hierarchical_clustering
* データ数を$n$とすると、何の工夫もしなければ時間計算量$O(n^3)$。
* なので、階層的クラスタリングは使わなくてよい。



---



## B) `MiniBatchKMeans`を試してみる

* 大規模データを扱うためには、ミニバッチ方式で動くアルゴリズムがあれば、それを使うのが良いだろう。
* 幸い、k-平均法には、ミニバッチ版がある。
* これにより、データをミニバッチとして少しずつ与えて、クラスタリングをすこしずつ進めることができる。
* そこで、ミニバッチ式のクラスタリングのについて、途中経過を調べてみる。

### `partial_fit`の実行を反復してみる

* バッチサイズは1000にしてみる。

In [ ]:
from sklearn.cluster import MiniBatchKMeans

n_clusters = 100
kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=123, batch_size=1000, max_iter=1)

* まずは1回、partial_fitを実行。

In [ ]:
kmeans.partial_fit(vectors)

* 適当なクラスタ（ここではインデックスが30のクラスタ）の、重心に近い順に上位20語を表示させてみる。

In [ ]:
center = kmeans.cluster_centers_[30]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
print(' '.join(words[indices[:20]]))

* もう1回、partial_fitを実行。

In [ ]:
kmeans.partial_fit(vectors)
center = kmeans.cluster_centers_[30]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
print(' '.join(words[indices[:20]]))

* 少しクラスタの様子が変わっている。さらにもう1回、partial_fitを実行。

In [ ]:
kmeans.partial_fit(vectors)
center = kmeans.cluster_centers_[30]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
print(' '.join(words[indices[:20]]))

* やはり変化している。10回ループを回してみる。

In [ ]:
for i in range(10):
  kmeans.partial_fit(vectors)
  center = kmeans.cluster_centers_[30]
  indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
  print(' '.join(words[indices[:20]].tolist()))

* さらに10回、ループを回す。
 * ループを回すついでに、所属するクラスタが変化した単語がいくつあったかを、毎回表示させてみる。


In [ ]:
for i in range(10):
  prev_labels = kmeans.labels_
  kmeans.partial_fit(vectors)
  center = kmeans.cluster_centers_[30]
  indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
  print(' '.join(words[indices[:20]].tolist()))
  print(f'{(kmeans.labels_ != prev_labels).sum()}個の単語が別のクラスタへ移った。')

* このあたりで、クラスタの重心を保存しておく。
 * ちなみに、保存したクラスタの重心は、再度同じデータセット上でk-平均法を実行するときに、クラスタリングの初期値として使える。

In [ ]:
np.savetxt('MiniBatchKMeans_cluster_centers_{:d}.csv'.format(n_clusters), kmeans.cluster_centers_, delimiter=',')

* さらに30回ほど回してみる。

In [ ]:
for i in range(30):
  prev_labels = kmeans.labels_
  kmeans.partial_fit(vectors)
  center = kmeans.cluster_centers_[30]
  indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
  print(' '.join(words[indices[:20]].tolist()))
  print(f'{(kmeans.labels_ != prev_labels).sum()}個の単語が別のクラスタへ移った。')

* 本当は、すべての単語がクラスタリングの対象になるまで、ループを回す必要あり。

* おそらく、最低、（単語数）÷（ミニバッチサイズ）回は、回す必要があると思われる。

 * partial_fitメソッドの仕様を、みなさん調べておいてください。

### クラスタのサイズを調べる

* NumPyの配列に、いろいろな値が何回ずつ出てくるかを知るには、unique関数を使うと良い。

In [ ]:
unique, counts = np.unique(kmeans.labels_, return_counts=True)

In [ ]:
unique

In [ ]:
counts

* クラスタのインデックスをキーとし、そのサイズを値とする辞書を作る。

In [ ]:
size_dict = dict(zip(unique, counts))

* 辞書のエントリを、キーではなく値でソートする。

In [ ]:
sorted_clusters = [k for k, v in sorted(size_dict.items(), key=lambda item: item[1], reverse=True)]

In [ ]:
counts[sorted_clusters]

### サイズが最大のクラスタを調べる

In [ ]:
center = kmeans.cluster_centers_[sorted_clusters[0]]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
' '.join(words[indices[:100]].tolist())

* あまり統一感がない。クラスタ数100個が少なかった可能性もある。

### サイズが最小のクラスタを調べる

In [ ]:
center = kmeans.cluster_centers_[sorted_clusters[99]]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
' '.join(words[indices[:100]].tolist())

* 命令的な表現が多いようだ。

### サイズが中間的なクラスタを調べる

In [ ]:
center = kmeans.cluster_centers_[sorted_clusters[49]]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
' '.join(words[indices[:100]].tolist())

* かなり統一感がある。

### 適当なクラスターを調べてみる

In [ ]:
center = kmeans.cluster_centers_[sorted_clusters[39]]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
' '.join(words[indices[:100]].tolist())

* スポーツ関係っぽい。

## C) Dask-MLのspectral clusteringを試してみる

* Dask-MLについてはWebサイトを参照（私も詳しくは知りません・・・）。
 * https://ml.dask.org/

* scalableなspectral clustering（ただし近似が入る）が実装されているので、使ってみる。
 * https://ml.dask.org/clustering.html

### Daskをインストール

In [14]:
! pip install -U tornado
! pip install dask distributed --upgrade
! pip install dask_ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 423 kB 4.3 MB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires tornado~=5.1.0; python_version >= "3.0", but you have tornado 6.2 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.2 MB/s 
     |████████████████████████████████| 837 kB 31.1 MB/s 
     |████████████████████████████████| 596 kB 54.8 MB/s 
     |████████████████████████████████| 140 kB 55.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 1.3.0
    Uninstalling cloudpickle-1.3.0:
      Successfully uninstalled cloudpickle-1.3.0
  Attempting uninstall: dask
    Found existing installation: dask 2.12.0
    Uninstalling dask-2.12.0:
      Successfully uninstalled dask-2.12.0
  Attempting uninstall: distributed
    Found existing installation: distributed 1.25.3
    Uninstalling distributed-1.25.3:
      Successfully uninstalled distributed-1.25.3
ERROR: pip's dependency res

* 単語ベクトルのデータを取得し直す



In [1]:
import numpy as np
from gensim.models.word2vec import Word2Vec

model_path = 'word2vec.gensim.model'
model = Word2Vec.load(model_path)
wv = model.wv
words = list()
vectors = list()
for word in wv.vocab:
  words.append(word)
  vectors.append(wv.word_vec(word))
words = np.array(words)
vectors = np.array(vectors)

### spectral clusteringを実行する

In [2]:
import dask_ml.cluster

n_clusters = 100
clusterer = dask_ml.cluster.SpectralClustering(n_clusters=n_clusters, random_state=123,
                                               n_jobs=-1, n_components=100)

* クラスタリングの実行。
 * しばらく待つ。10分弱ぐらい。

In [3]:
clusterer.fit(vectors)

Found fewer than 100 clusters in init (found 24).


SpectralClustering(n_clusters=100, n_jobs=-1, random_state=123)

* ラベル（各データ点がどのクラスタに属するか）の情報のデータ型を調べる。

In [4]:
labels = clusterer.labels_
type(labels)

dask.array.core.Array

* Dask固有のデータ型のようなのでNumpyの配列に変換する。

In [5]:
labels = labels.compute()
type(labels)

numpy.ndarray

* ラベルをファイルに保存しておく。

In [6]:
np.save('dask_sc_labels', labels)

In [7]:
labels

array([ 0,  9, 12, ..., 71,  2, 90], dtype=int32)

* クラスタのサイズ順にクラスタのインデックス（0~99）をソートする。

In [8]:
unique, counts = np.unique(labels, return_counts=True)
size_dict = dict(zip(unique, counts))
sorted_clusters = [k for k, v in sorted(size_dict.items(), key=lambda item: item[1], reverse=True)]

### クラスタのサイズを調べる

In [9]:
counts[sorted_clusters]

array([8282, 5764, 4985, 4824, 4809, 4777, 4769, 4704, 4552, 4535, 4491,
       4468, 4435, 4387, 4369, 4261, 4221, 4215, 4200, 4083, 4082, 4002,
       3990, 3916, 3847, 3841, 3822, 3790, 3769, 3726, 3712, 3708, 3656,
       3642, 3587, 3577, 3537, 3530, 3528, 3511, 3488, 3462, 3451, 3425,
       3419, 3376, 3360, 3342, 3324, 3321, 3235, 3233, 3187, 3177, 3166,
       3160, 3136, 3125, 3122, 3088, 3087, 3086, 3086, 3065, 3056, 3045,
       3031, 2945, 2944, 2914, 2883, 2864, 2852, 2812, 2791, 2755, 2740,
       2728, 2703, 2702, 2649, 2633, 2530, 2445, 2383, 2371, 2310, 2284,
       2283, 2221, 2170, 2115, 2111, 2079, 2073, 1979, 1936, 1349, 1298,
        667])

### サイズが最大のクラスタを調べる

In [10]:
' '.join(words[labels == sorted_clusters[0]])

'松平正容 カイム Martina bx +∞ ユニットコム まどか☆マギカ マインド・ゲーム ポルデノーネ無声映画祭 六神合体ゴッドマーズ サンダーマスク ワンサくん プラレス3四郎 機甲猟兵メロウリンク 15.2m 忍者戦隊カクレンジャー 超獣戦隊ライブマン 魔界村 宇宙空母ギャラクティカ especially フランケンシュタインの逆襲 日本国憲法の改正手続に関する法律 6120 シークレット・ドクトリン ~、 ヤッピー LESSON ガンパウダー プリンスオブウェールズ 宮城県警察部 マエストラーレ 東海艦隊 シレ 昭洋 おりおん ゴーディ・ハウ 世界フィギュアスケート国別対抗戦 33打点 熊姫 ザ・イロモネア 丹青 世界の果てまで セブン・アークス ネレウス Tatiana ラジオどっとあい ヒューマ コミックフラッパー 小高駅 新地駅 鬼武者 バイオハザード2 RSIP ACD 公証人法 種田仁 片岡我童 初代歌川豊国 黒部の太陽 餝 Outrage のろう 引きおこし ゴー・アヘッド・イーグルス パワプロクンポケット ハンゲ 王立宇宙軍 オネアミスの翼 フリクリ 福岡県立小倉高等学校 滋賀県立彦根東高等学校 エンガノ Sumner 超時空世紀オーガス 超時空騎団サザンクロス 機甲創世記モスピーダ 大鉄人17 超時空要塞マクロスII -LOVERS AGAIN- ルイジ・コメンチーニ 一興 少年チャンピオン わしズム 予防外交 梶岡定道 ジム・コマンド ガンダムウォー 機動戦士ガンダム 戦場の絆 ドラゴンバスター サラマンドラ パトリック・ラフター アルノー・クレマン Cochran EC2 Florent ハチミツとクローバー ベルヌ 若きウェルテルの悩み 3720 朝夜 センター南 松本電鉄バス 定州市 月刊電撃コミックガオ! 別冊ヤングマガジン 特警ウインスペクター 特捜エクシードラフト 4コマまんが王国 スーパードンキーコング {(" Grosso 五星戦隊ダイレンジャー 宇宙をかける少女 子鹿物語 デュアルヘッド hydroxide gsi 母をたずねて三千里 1583 フレデリック・ミストラル ベール湖 メトロポール・ド・リヨン 0020 府中藩 徳川宗翰 晴れの海 アトランテFC エクゼター Mandarin 松平頼純 ゆでん 大阪ゴールド

### サイズが最小のクラスタを調べる

In [11]:
' '.join(words[labels == sorted_clusters[99]])

'求めよ 連れ戻そ だろ しよ ご 戻ろ 盗も あろ 受け取ろ 抑えよ 登ろ けんど 戻そ 取り持と つこ させよ 押し通そ せよ 示そ 集めよ こんど 出そ 討と 高めよ 行なお 語ろ 納めよ 言お 渡そ 及ぼ でしょ 立と 抜こ 斬ろ やろ 付けよ 見よ 殺そ 収めよ 立ち去ろ 引き返そ 慰めよ 切ろ いえよ 返そ もらお ましょ 帰ろ 見極めよ 殴ろ 探ろ してやろ かかろ まいと さそ 奪お あげよ 聞こ 刺そ 買お 捕らえよ 送ろ 贈ろ 戦お ちろ 備えよ ぶろ 迎えよ ほしょ えんど 決めよ 伝えよ 知らせよ 遊ぼ かいきょ 受けよ かんす 出よ いこ おこ 立て直そ かわそ 渡ろ たろ 保と かけよ 与えよ 役立てよ 行お 得よ 逃げ出そ 助けよ 近づこ 合わせよ ていりゅうじょ 洗お みよ 始まろ 図ろ 彷徨 届けよ 減らそ 辞めよ なろ 立てよ られよ 起こそ 封じよ 結ぼ 調べよ 救お ごまかそ 避けよ 抱きしめよ 防ご 歌お 守ろ 上げよ 導こ 述べよ 割ろ 打と 死の いよ 見せよ よかろ 取り戻そ 売り出そ 変えよ 目指そ 逃げよ 乗ろ 言えよ 妨げよ 知ろ 乗り切ろ 作り上げよ 育てよ 入れよ 認めよ 及ぼそ 加えよ 作ろ しじょ 乗り込も 留めよ 尽くそ 働こ 続けよ 組み込も 潰そ 広げよ 挑も 探そ げんこ 進も 座ろ 掴も ぎゅ 見出そ つけよ 書こ 倒そ 見直そ 開こ 追い払お 乗っ取ろ 用いよ 譲ろ 行こ 追い出そ れよ 説こ 絶する 置こ づけよ 笑お 盛り上げよ 食お やめよ じゃろ できよ 狙お 止めよ ぶそ 盛り込も ぶしゅ 葬ろ 通り過ぎよ 取ろ 始めよ 捨てよ 応えよ 逃れよ 補お 握ろ 進めよ 強めよ 当たろ 起きよ 壊そ 描こ 採ろ 貫こ 持と ありか 使お 極めよ 抜け出そ 果して 消そ 会お きんじろ 出かけよ 召し出そ ぎじょ とうじょ 貸そ 触ろ げつよ 仕立てよ 覚えよ 明かそ 済ませよ 稼ご 去ろ 貰お 捕まえよ 滅ぼそ 逃がそ 移そ 確かめよ っしょ 消えよ つくろ むそ 退けよ 就こ 封じ込めよ 暴こ 応じよ 仕えよ 定めよ 支えよ 攻めよ 考えよ 話そ 攻め込も 引こ 掛けよ 飛び込も 動かそ 隠そ 伸ばそ 持ち出そ はかろ もたらそ 売り込も 裂こ 引き出そ 挙げよ 広めよ 取り上げよ 深め

### サイズが中間的なクラスタを調べる

In [12]:
' '.join(words[labels == sorted_clusters[49]])

'称号 1675年 1680年 1731年 1740年 1750年 王子 司教 紋章 1804年 1805年 1806年 1822年 1825年 1846年 1827年 1852年 1847年 1813年 1816年 男系 特権 ザイン 2世 パッサウ オーストリア 聖 シュテファン 大聖堂 アウクスブルクの和議 1855年 1786年 1854年 レオン フランス 死後 ヴィルヘルム ドルスキニンカイ ポーランド人 貴族 公爵 ワルシャワ アレクサンドル ＝ 1863年 公 もとで ブルゴーニュ公国 ハプスブルク家 1814年 1830年 ルクセンブルク大公国 同君連合 朝 1世 王 イングランド 王位 1707年 国王 フェリペ 5世 王妃 サヴォイア ヴィットーリオ・アメデーオ 公女 マリア ルイーザ フェルナンド 即い サンチョ Sancho アラゴン王国 ナバラ アラゴン ラミロ 4世 カスティーリャ王 アルフォンソ 世 即位 カスティーリャ レオン王国 ラ王 バルバストロ ペドロ1世 伯 3世 イサベル 1662年 1679年 1682年 1718年 1832年 ハンガリー 男爵 ピピン 1522年 国教会 卿 王女 ベレンガリア イングランド王 リチャード1世 妃 ブランカ シャンパーニュ ティボー テオバルド 1714年 デヴォンシャー 1610年 1721年 1723年 1722年 1788年 1831年 1840年 フィリップ 1630年 1748年 1811年 南フランス ミラノ 1784年 伯爵 1809年 1761年 君主 王族 クレオパトラ イギリスの君主 ピョートル大帝 皇帝 ケルン ルイ13世 枢機卿 リシュリュー ジャンヌ・ダルク ミラボー フランス革命 エカチェリーナ2世 家長 ポーランド立憲王国 1815年 ナポレオン・ボナパルト ワルシャワ公国 ポーランド王国 ロシア皇帝 ナポレオン クレメンス メッテルニヒ ニコライ デカブリストの乱 1848年 農奴解放令 シュラフタ スウェーデン＝ノルウェー カール ニコライ2世 退位 ロマノフ朝 リヨン ニース 祝う 1767年 1715年 1812年 1834年 1775年 1794年 プロヴァンス ジャ ユダヤ教徒 アル 1771年 1819年 1795年 1808年 ローマ王 ロ

* 全てのクラスタについて、ランダムな50語を表示

In [20]:
for i in range(len(sorted_clusters)):
  members = words[labels == sorted_clusters[i]]
  np.random.shuffle(members)
  print(' '.join(members[:50]))
  print('-'*80)

フラッシュマン 日本ミステリー文学大賞新人賞 ザ・サンデー 金色のガッシュ!! 山本浩 サニングデール 国立大学協会 後宮からの誘拐 カッコーの巣の上で 工業意匠 シカゴ商品取引所 西保 ジョン・クランコ 彼氏彼女の事情 オオカミ少年 ネイチャーゲーム トータル・イクリプス ラグラジアン 静岡中学校 部垂 86打点 箱島 トラベルトレイン 自然対数の底 アムンゼン フォックス・テリア 夫神 ミヒャエル・ハイドン プチヴリ 臨時目的放送 なめ猫 2234 グリーンメディア 阜新市 国際平和協力本部 Psyche リュドミーラ・ジュラヴリョーワ Squalus スティーブニッジ 宴のあと メンカウラー王 久貝 ヴィボ・ヴァレンツィア インデペンディエンテ・メデジン パシフィック・リム 谷本貴義 マルタイ 2231 リンコサウルス 2133
--------------------------------------------------------------------------------
フォーミュラワン スィヤードヴァーダ ベルリヒンゲン HORC 14496 ピロピロ 救難員 競泳水着 藤枝順心 カーシージ カルネアデス 宛平県 SSゾイド ナメラ 掖門 大フーガ シンオウ ファーストステップ 名古屋おもてなし武将隊 57年ぶり ネオテニー GCSB Convoy 1557 台風17号 南充市 大阪10区 ジャック・ホーナー 米韓連合司令部 タノーム アステリオン 代替放送 マデルノ ゴシックロリータ BOURBON サイドスラスター ファースト・クラス めでたし クルチャトフ テルカ 技能労働者 シンプレックス ダンテス マルロー 過酸化ベンゾイル 玉ノ井部屋 ロクテ ジェイムズ＝ヤンガー・ギャング 国道270号 第34戦
--------------------------------------------------------------------------------
へんせい くり ろくし ひっと くさい とっと すたー かいしん かせ まりか かんぽ ちちゅう あゆか つるおか ねむり きじゅ はむ けつ きわ らち くうぼ かえ きちょう しり みえ はなよめ かいまく ともよ びほう やまと わー おせ さいわい れき らむ えいじ